カリフォルニア住宅価格データセットに対し、ランダムフォレスト回帰モデルを構築。
RandomizedSearchCVでハイパーパラメータ調整後、最適なモデルで価格を予測し、精度を評価。
RandomizedSearchCVにより適切なパラメータの組み合わせを探索。

In [1]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint  # 整数値のランダム分布用

# データセットのロードと前処理
california_housing = fetch_california_housing()
df = pd.DataFrame(california_housing.data, columns=california_housing.feature_names)
df['MedHouseVal'] = california_housing.target
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ハイパーパラメータの範囲を確率分布で設定
param_distributions = {
    'n_estimators': randint(10, 100),  # 10から100までのランダムな整数
    'max_depth': [None] + list(randint(10, 30).rvs(10)),  # Noneと10から30までのランダムな整数10個
    'min_samples_split': randint(2, 10)  # 2から10までのランダムな整数
}

# ランダムフォレストモデルの作成
model = RandomForestRegressor(random_state=42)

# RandomizedSearchCVによるハイパーパラメータチューニング
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    n_iter=100,  # 試行回数
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42
)

# 学習の実行
random_search.fit(X_train, y_train)

# 最適なハイパーパラメータとスコアを表示
print("Best parameters:", random_search.best_params_)
print("Best score:", random_search.best_score_)

# 最適なモデルでテストデータを予測
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# モデルの評価
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test MSE: {mse:.4f}, Test R2: {r2:.4f}")

Best parameters: {'max_depth': 22, 'min_samples_split': 5, 'n_estimators': 98}
Best score: -0.26103753911075805
Test MSE: 0.2556, Test R2: 0.8049
